In [1]:
import torch 
import pandas as pd
from sklearn.preprocessing import OneHotEncoder as ohe
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm


In [2]:
df = pd.read_csv("../../data/raw/obesity_prediction.csv")

In [3]:
df.head(3)

,Gender,Age,Height,Weight,family_history,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,Obesity
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight


In [4]:
enc = ohe(handle_unknown="warn",sparse_output=False).set_output(transform="pandas")

In [5]:
enc.fit(np.asarray(df["Gender"]).reshape(-1,1))

OneHotEncoder(handle_unknown='warn', sparse_output=False)

In [6]:
enc.categories_

[array(['Female', 'Male'], dtype=object)]

In [7]:
gender_enc_df = enc.transform(np.asarray(df["Gender"]).reshape(-1,1))

In [8]:
gender_enc_df

,x0_Female,x0_Male
0,1.0,0.0
1,1.0,0.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0
...,...,...
2106,1.0,0.0
2107,1.0,0.0
2108,1.0,0.0
2109,1.0,0.0


In [9]:
gender_enc_df.columns =["female","male"]

In [10]:
gender_enc_df.head()

,female,male
0,1.0,0.0
1,1.0,0.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0


In [11]:
enc2 = ohe(sparse_output=False,handle_unknown="ignore").set_output(transform="pandas")

In [12]:
df_mtran = enc2.fit_transform(np.asarray(df["MTRANS"]).reshape(-1,1))

In [13]:
df_mtran.head()

,x0_Automobile,x0_Bike,x0_Motorbike,x0_Public_Transportation,x0_Walking
0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0,0.0


In [14]:
enc2.categories_

[array(['Automobile', 'Bike', 'Motorbike', 'Public_Transportation',
        'Walking'], dtype=object)]

In [15]:
df_mtran.columns = ["automobile","bike","motorbike","public_transportation","walking"]

In [16]:
df_mtran.head(3)

,automobile,bike,motorbike,public_transportation,walking
0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0


In [17]:
df_mtran.head()

,automobile,bike,motorbike,public_transportation,walking
0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0,0.0


In [18]:
gender_enc_df.head()

,female,male
0,1.0,0.0
1,1.0,0.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0


In [19]:
df_x = pd.concat(objs=[gender_enc_df,df_mtran,df["Age"],df["Height"]],axis=1)

In [20]:
df_x

,female,male,automobile,bike,motorbike,public_transportation,walking,Age,Height
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,21.000000,1.620000
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,21.000000,1.520000
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,23.000000,1.800000
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,27.000000,1.800000
4,0.0,1.0,0.0,0.0,0.0,1.0,0.0,22.000000,1.780000
...,...,...,...,...,...,...,...,...,...
2106,1.0,0.0,0.0,0.0,0.0,1.0,0.0,20.976842,1.710730
2107,1.0,0.0,0.0,0.0,0.0,1.0,0.0,21.982942,1.748584
2108,1.0,0.0,0.0,0.0,0.0,1.0,0.0,22.524036,1.752206
2109,1.0,0.0,0.0,0.0,0.0,1.0,0.0,24.361936,1.739450


In [21]:
from sklearn.preprocessing import LabelEncoder

In [22]:
le = LabelEncoder()

In [23]:
y_enc = le.fit_transform(df["Obesity"])

In [24]:
y_enc

array([1, 1, 1, ..., 4, 4, 4], shape=(2111,))

In [25]:
le.classes_

array(['Insufficient_Weight', 'Normal_Weight', 'Obesity_Type_I',
       'Obesity_Type_II', 'Obesity_Type_III', 'Overweight_Level_I',
       'Overweight_Level_II'], dtype=object)

In [26]:
Y = torch.tensor(y_enc)

In [27]:
Y.shape

torch.Size([2111])

In [28]:
scaler = StandardScaler().set_output(transform="pandas")

In [29]:
df_age_scaled = scaler.fit_transform(np.asarray(df["Age"]).reshape(-1,1))
df_height_scaled = scaler.fit_transform(np.asarray(df["Height"]).reshape(-1,1))

In [30]:
df_height_scaled.columns =["height"]
df_age_scaled.columns = ["age"]

In [31]:
df_height_scaled

,height
0,-0.875589
1,-1.947599
2,1.054029
3,1.054029
4,0.839627
...,...
2106,0.097045
2107,0.502844
2108,0.541672
2109,0.404927


In [32]:
df_x = df_x.drop(labels=["Age","Height"],axis=1)
df_x.head()

,female,male,automobile,bike,motorbike,public_transportation,walking
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [33]:
df_x = pd.concat(axis=1,objs=[df_x,df_age_scaled,df_height_scaled])

In [34]:
df_x.head()

,female,male,automobile,bike,motorbike,public_transportation,walking,age,height
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.522124,-0.875589
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.522124,-1.947599
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,-0.206889,1.054029
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.423582,1.054029
4,0.0,1.0,0.0,0.0,0.0,1.0,0.0,-0.364507,0.839627


In [35]:
class ffn(torch.nn.Module):
    
    def __init__(self,input_dims):
        super(ffn,self).__init__()    
        
        self.relu    = torch.nn.ReLU()
        self.sigmoid = torch.nn.Sigmoid()
        self.tanh    = torch.nn.Tanh()
        self.layer1  = torch.nn.Linear(input_dims,70)
        self.layer2  = torch.nn.Linear(70,70)
        self.layer3  = torch.nn.Linear(70,30)
        self.layer4  = torch.nn.Linear(30,7)
    def forward(self,input):
        
        out = self.tanh(self.layer1(input))
        out = self.tanh(self.layer2(out))
        out = self.tanh(self.layer3(out))
        out = self.layer4(out)
        return out

In [36]:
Y

tensor([1, 1, 1,  ..., 4, 4, 4])

In [37]:
X = torch.tensor(np.asarray(df_x))

In [38]:
X.shape

torch.Size([2111, 9])

In [39]:
df_x.head()

,female,male,automobile,bike,motorbike,public_transportation,walking,age,height
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.522124,-0.875589
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.522124,-1.947599
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,-0.206889,1.054029
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.423582,1.054029
4,0.0,1.0,0.0,0.0,0.0,1.0,0.0,-0.364507,0.839627


In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [41]:
class data(torch.utils.data.Dataset):

    def __init__(self, X,y):
        self.x = X
        self.y = y
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        record = self.x[index]
        label = self.y[index]
        return record,label

In [42]:
len(X_train[0])

9

In [43]:
model = ffn(9)

In [44]:
train_dataset = data(X_train,y_train)
test_dataset = data(X_test,y_test)

train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=32,shuffle=True)

In [45]:
model.train()
epochs = 200

#optim = torch.optim.SGD(model.parameters(),lr = 0.1,momentum=.7)
optim = torch.optim.AdamW(model.parameters(),lr = 0.001)
loss_fn = torch.nn.CrossEntropyLoss()
y_true = []
y_pred = []

for epoch in range(epochs):
    loss_per_epoch = 0
    for batch in tqdm(train_loader):
        
        inputs,labels = batch
        optim.zero_grad()
        
        outputs = model(inputs.float())
        loss    = loss_fn(outputs,labels)
       
        _,preds = torch.max(outputs.detach(),1)
    
        y_pred.extend(preds.detach().tolist())
        y_true.extend(labels.detach().tolist())
        loss.backward()
        optim.step()
        
        loss_per_epoch = loss.item()
    
    print(f"Epoch: {epoch + 1}   Loss: {loss_per_epoch}")
    print(classification_report(y_true,y_pred,zero_division=0))

100%|██████████| 45/45 [00:00<00:00, 835.36it/s]


Epoch: 1   Loss: 1.8614563941955566
              precision    recall  f1-score   support

           0       0.15      0.10      0.12       182
           1       0.21      0.03      0.06       184
           2       0.00      0.00      0.00       237
           3       0.31      0.91      0.46       203
           4       0.32      0.91      0.48       209
           5       0.06      0.01      0.02       196
           6       0.21      0.03      0.06       203

    accuracy                           0.29      1414
   macro avg       0.18      0.29      0.17      1414
weighted avg       0.18      0.29      0.17      1414



100%|██████████| 45/45 [00:00<00:00, 910.60it/s]


Epoch: 2   Loss: 1.623302936553955
              precision    recall  f1-score   support

           0       0.25      0.15      0.19       364
           1       0.21      0.07      0.10       368
           2       0.43      0.03      0.06       474
           3       0.33      0.94      0.49       406
           4       0.36      0.96      0.53       418
           5       0.24      0.08      0.12       392
           6       0.29      0.07      0.11       406

    accuracy                           0.33      2828
   macro avg       0.30      0.33      0.23      2828
weighted avg       0.31      0.33      0.23      2828



100%|██████████| 45/45 [00:00<00:00, 888.93it/s]


...

Epoch: 198   Loss: 0.6447883248329163
              precision    recall  f1-score   support

           0       0.53      0.64      0.58     36036
           1       0.52      0.32      0.39     36432
           2       0.57      0.62      0.59     46926
           3       0.71      0.89      0.79     40194
           4       0.75      0.94      0.84     41382
           5       0.56      0.43      0.49     38808
           6       0.62      0.47      0.53     40194

    accuracy                           0.62    279972
   macro avg       0.61      0.62      0.60    279972
weighted avg       0.61      0.62      0.61    279972



100%|██████████| 45/45 [00:00<00:00, 885.70it/s]


Epoch: 199   Loss: 0.8626670241355896
              precision    recall  f1-score   support

           0       0.53      0.64      0.58     36218
           1       0.52      0.32      0.40     36616
           2       0.57      0.62      0.59     47163
           3       0.71      0.89      0.79     40397
           4       0.75      0.94      0.84     41591
           5       0.56      0.43      0.49     39004
           6       0.62      0.47      0.53     40397

    accuracy                           0.62    281386
   macro avg       0.61      0.62      0.60    281386
weighted avg       0.61      0.62      0.61    281386



100%|██████████| 45/45 [00:00<00:00, 940.76it/s]


Epoch: 200   Loss: 1.0198200941085815
              precision    recall  f1-score   support

           0       0.53      0.64      0.58     36400
           1       0.53      0.32      0.40     36800
           2       0.57      0.62      0.59     47400
           3       0.71      0.89      0.79     40600
           4       0.75      0.94      0.84     41800
           5       0.56      0.44      0.49     39200
           6       0.62      0.47      0.53     40600

    accuracy                           0.62    282800
   macro avg       0.61      0.62      0.60    282800
weighted avg       0.61      0.62      0.61    282800



In [46]:
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    test_loss = 0
    for batch in test_loader:
        
        inputs,labels = batch
        outputs = model(inputs.float())
        loss = loss_fn(outputs,labels)
        test_loss = loss.item()
        _,preds = torch.max(outputs.detach(),1)
        y_pred.extend(preds.detach().tolist())
        y_true.extend(labels.detach().tolist())

    print(f"Test Loss: {test_loss}")
    print(classification_report(y_true,y_pred,zero_division=0))
    
    

Test Loss: 0.6707621216773987
              precision    recall  f1-score   support

           0       0.51      0.70      0.59        90
           1       0.55      0.27      0.36       103
           2       0.58      0.61      0.60       114
           3       0.70      0.86      0.77        94
           4       0.84      0.94      0.89       115
           5       0.51      0.48      0.49        94
           6       0.53      0.41      0.46        87

    accuracy                           0.62       697
   macro avg       0.60      0.61      0.59       697
weighted avg       0.61      0.62      0.60       697



In [47]:
dict = {0:0,1:0,2:0,3:0,4:0,5:0,6:0}

for cls in y_train:

    y = cls.item()

    dict[y] += 1